### レースデータを取得し登録するコード

基本的に完全に動いている！！

In [1]:
import sys
import os
import logging
from datetime import datetime, date
import pandas as pd
import json

# プロジェクトのルートディレクトリをパスに追加
sys.path.append('..')

# ログ設定
logging.basicConfig(
    level=logging.WARNING, # WARNING以上のみ表示（INFO, DEBUGを非表示）
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("=== 競馬レースデータ収集システム ===")
print(f"実行時刻: {datetime.now()}")


=== 競馬レースデータ収集システム ===
実行時刻: 2025-08-31 22:27:31.981230


In [2]:
# 必要なモジュールをインポート
from src.scraping.scrapers.race_scraper import RaceScraper
from src.scraping.storage.race_storage import RaceStorage
from src.database.schemas.race_schema import Race, RaceResult
from src.scraping.storage.postgresql_storage import PostgreSQLStorage

# スクレイパーとストレージを初期化
scraper = RaceScraper(delay=1.0)  # 1秒間隔でリクエスト
storage = RaceStorage()

print("✅ システム初期化完了")

✅ システム初期化完了


In [3]:
# レースリストを取得
print("=== Testing Race List Extraction ===")
race_list = scraper.scrape_race_list_by_conditions(
    start_year=2010,
    end_year=2018,
    grades=['1'],
    limit=140
)

print(f"Found {len(race_list)} races")
for race in race_list[:3]:  # 最初の3件のみ表示
    print(f"  - {race}")

2025-08-31 22:27:37,071 - src.scraping.scrapers.race_scraper - INFO - Fetching race list: 2010-2018, grades: ['1'], limit: 140
2025-08-31 22:27:37,072 - src.scraping.scrapers.race_scraper - INFO - Fetching page 1...


=== Testing Race List Extraction ===


2025-08-31 22:27:38,997 - src.scraping.extractors.race.race_list_extractor - INFO - Extracted 100 races from list
2025-08-31 22:27:38,997 - src.scraping.scrapers.race_scraper - INFO - Page 1: Found 100 races
2025-08-31 22:27:40,001 - src.scraping.scrapers.race_scraper - INFO - Fetching page 2...
2025-08-31 22:27:41,225 - src.scraping.extractors.race.race_list_extractor - INFO - Extracted 84 races from list
2025-08-31 22:27:41,226 - src.scraping.scrapers.race_scraper - INFO - Page 2: Found 84 races
2025-08-31 22:27:41,226 - src.scraping.scrapers.race_scraper - INFO - Total races collected: 140


Found 140 races
  - {'race_name': 'ホープフルS(GI)', 'race_id': '201806050911'}
  - {'race_name': '有馬記念(GI)', 'race_id': '201806050811'}
  - {'race_name': '朝日フューチュリティ(GI)', 'race_id': '201809050611'}


In [4]:
# 取得できたレース名のみを全て羅列
print("✅ レースリスト取得完了")

race_names = [race['race_name'] for race in race_list]
print(" | ".join(race_names))

✅ レースリスト取得完了
ホープフルS(GI) | 有馬記念(GI) | 朝日フューチュリティ(GI) | 阪神ジュベナイルF(GI) | ジャパンC(GI) | マイルチャンピオンS(GI) | エリザベス女王杯(GI) | 天皇賞(秋)(GI) | 菊花賞(GI) | 秋華賞(GI) | スプリンターズS(GI) | 宝塚記念(GI) | 安田記念(GI) | 東京優駿(GI) | 優駿牝馬(GI) | ヴィクトリアマイル(GI) | NHKマイルC(GI) | 天皇賞(春)(GI) | 皐月賞(GI) | 桜花賞(GI) | 大阪杯(GI) | 高松宮記念(GI) | ホープフルS(GI) | 有馬記念(GI) | 朝日フューチュリティ(GI) | 阪神ジュベナイルF(GI) | ジャパンC(GI) | マイルチャンピオンS(GI) | エリザベス女王杯(GI) | 天皇賞(秋)(GI) | 菊花賞(GI) | 秋華賞(GI) | スプリンターズS(GI) | 宝塚記念(GI) | 安田記念(GI) | 東京優駿(GI) | 優駿牝馬(GI) | ヴィクトリアマイル(GI) | NHKマイルC(GI) | 天皇賞(春)(GI) | 皐月賞(GI) | 桜花賞(GI) | 大阪杯(GI) | 高松宮記念(GI) | 有馬記念(GI) | 朝日フューチュリティ(GI) | 阪神ジュベナイルF(GI) | ジャパンC(GI) | マイルチャンピオンS(GI) | エリザベス女王杯(GI) | 天皇賞(秋)(GI) | 菊花賞(GI) | 秋華賞(GI) | スプリンターズS(GI) | 宝塚記念(GI) | 安田記念(GI) | 東京優駿(GI) | 優駿牝馬(GI) | ヴィクトリアマイル(GI) | NHKマイルC(GI) | 天皇賞(春)(GI) | 皐月賞(GI) | 桜花賞(GI) | 高松宮記念(GI) | 有馬記念(GI) | 朝日フューチュリティ(GI) | 阪神ジュベナイルF(GI) | ジャパンC(GI) | マイルチャンピオンS(GI) | エリザベス女王杯(GI) | 天皇賞(秋)(GI) | 菊花賞(GI) | 秋華賞(GI) | スプリンターズS(GI) | 宝塚記念(GI) | 安田記念(GI) | 東京優駿(GI) | 優駿牝馬(GI

In [5]:
# レース詳細と結果を取得して保存
storage = PostgreSQLStorage()

# race_list = race_list[:3]  # テストのため3件に制限

for race in race_list:
    race_detail, race_results = scraper.scrape_race_detail(race['race_id'])

    print(f"現在処理中のレース: {race_detail}")

    storage.insert_race(race_detail)
    storage.insert_race_results(race_results)

2025-08-31 22:31:01,247 - src.scraping.storage.postgresql_storage - INFO - PostgreSQL connection test successful
2025-08-31 22:31:01,249 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201806050911/
2025-08-31 22:31:01,624 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 13 race results for 201806050911
2025-08-31 22:31:01,625 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201806050911 (13 horses)
2025-08-31 22:31:01,625 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201806050911', race_date=datetime.date(2018, 12, 28), track_name='中山', race_number=11, race_name='第35回ホープフルステークス(GI)', distance=2000, track_type='芝', total_horses=13, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 30), winning_time='2:01.6', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31

現在処理中のレース: Race(race_id='201806050911', race_date=datetime.date(2018, 12, 28), track_name='中山', race_number=11, race_name='第35回ホープフルステークス(GI)', distance=2000, track_type='芝', total_horses=13, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 30), winning_time='2:01.6', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 22:31:02,824 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 16 race results for 201806050811
2025-08-31 22:31:02,825 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201806050811 (16 horses)
2025-08-31 22:31:02,825 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201806050811', race_date=datetime.date(2018, 12, 23), track_name='中山', race_number=11, race_name='第63回有馬記念(GI)', distance=2500, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 25), winning_time='2:32.2', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31 22:31:02,959 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201809050611/


現在処理中のレース: Race(race_id='201806050811', race_date=datetime.date(2018, 12, 23), track_name='中山', race_number=11, race_name='第63回有馬記念(GI)', distance=2500, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 25), winning_time='2:32.2', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 22:31:03,662 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 15 race results for 201809050611
2025-08-31 22:31:03,662 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201809050611 (15 horses)
2025-08-31 22:31:03,663 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201809050611', race_date=datetime.date(2018, 12, 16), track_name='阪神', race_number=11, race_name='第70回朝日フューチュリティ(GI)', distance=1600, track_type='芝', total_horses=15, grade='G1', track_direction='右', weather='小雨', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:33.9', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31 22:31:03,823 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201809050411/


...

現在処理中のレース: Race(race_id='201205021011', race_date=datetime.date(2012, 5, 20), track_name='東京1', race_number=11, race_name='第73回優駿牝馬(GI)', distance=2400, track_type='芝', total_horses=18, grade='G1', track_direction='左', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='2:23.6', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 22:32:55,460 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 18 race results for 201205020811
2025-08-31 22:32:55,460 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201205020811 (18 horses)
2025-08-31 22:32:55,461 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201205020811', race_date=datetime.date(2012, 5, 13), track_name='東京', race_number=11, race_name='第7回ヴィクトリアマイル(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='左', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:32.4', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


現在処理中のレース: Race(race_id='201205020811', race_date=datetime.date(2012, 5, 13), track_name='東京', race_number=11, race_name='第7回ヴィクトリアマイル(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='左', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:32.4', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 22:32:55,686 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201205020611/
2025-08-31 22:32:56,427 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 18 race results for 201205020611
2025-08-31 22:32:56,427 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201205020611 (18 horses)
2025-08-31 22:32:56,428 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201205020611', race_date=datetime.date(2012, 5, 6), track_name='東京', race_number=11, race_name='第17回NHKマイルカップ(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='左', weather='曇', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:34.5', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


現在処理中のレース: Race(race_id='201205020611', race_date=datetime.date(2012, 5, 6), track_name='東京', race_number=11, race_name='第17回NHKマイルカップ(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='左', weather='曇', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:34.5', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
